In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

from datasets import Dataset, DatasetDict
from transformers import AutoTokenizer, AutoModelForSequenceClassification
from transformers import Trainer, TrainingArguments

import pandas as pd
import numpy as np
import torch
import json

# WangchanBERTa tutorial 

https://colab.research.google.com/drive/1Kbk6sBspZLwcnOE61adAQo30xxqOQ9ko#scrollTo=r_IY0lDi1qTp

## Load raw data

In [ ]:
df = pd.read_csv("./20210714_slim_annotation.csv")
df = df[df['label'].notna() & df["comment_text_cleaned"].notna()]

df["label"] =  df["label"].astype(int)

In [ ]:
df.shape

In [ ]:
df.groupby("label").count()

In [ ]:
train_df, val_df = train_test_split(df, test_size=0.2, random_state=112)

## Load Model and Tokenizer

In [ ]:
model_name = "airesearch/wangchanberta-base-att-spm-uncased"
#revision = "finetuned@wisesight_sentiment"
revision = None
model = AutoModelForSequenceClassification.from_pretrained(
    model_name, num_labels=2, revision=revision
)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained(
                model_name,
                revision='main',
                model_max_length=416,)

## Create train/val Dataset

In [ ]:
class SalimDataset(torch.utils.data.Dataset):
    
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
train_texts = train_df["comment_text_cleaned"].tolist()
val_texts = val_df["comment_text_cleaned"].tolist()


train_labels = train_df["label"].tolist()
val_labels = val_df["label"].tolist()

In [ ]:
train_encodings = tokenizer(train_texts, truncation=True, padding=True)
val_encodings = tokenizer(val_texts, truncation=True, padding=True)

In [ ]:
train_dataset = SalimDataset(train_encodings, train_labels)
val_dataset = SalimDataset(val_encodings, val_labels)

In [ ]:
print(len(train_dataset))
print(len(val_dataset))

## Training Parameter

In [ ]:
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds, average="macro")
    acc = accuracy_score(labels, preds)
    
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }

In [ ]:
training_args = TrainingArguments(
    output_dir='./salim_classification',          # output directory
    num_train_epochs=10,              # total number of training epochs
    per_device_train_batch_size=8,  # batch size per device during training
    per_device_eval_batch_size=8,   # batch size for evaluation
    warmup_steps=500,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./salim_classification_logs',            # directory for storing logs
    logging_steps=10,
    #evaluation_strategy='epoch'
)


In [ ]:
trainer = Trainer(
    model=model,                         # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
    eval_dataset=val_dataset,        # evaluation dataset
    compute_metrics=compute_metrics,
)

In [ ]:
trainer.train()

In [ ]:
result_eval = trainer.evaluate()

In [ ]:
result_eval

In [ ]:
print(json.dumps(result_eval, indent=4))

In [ ]:
save_path = "./models/10/"
model.save_pretrained(save_path)
tokenizer.save_pretrained(save_path)

# Prediction

In [ ]:
from torch.nn.functional import softmax

In [ ]:
text = "รัฐรับผิดชอบทุกชีวิตไม่ได้หรอกคนให้บริการต้องจัดการเองถ้าจะเปิดผับบาร์"

device = "cuda"

_inputs = tokenizer(text, return_tensors="pt").to(device)

outputs = model(**_inputs)

outputs

In [ ]:
softmax(outputs[0], dim=1).cpu().data.numpy().round(3)